### requests.get을 이용한 웹 페이지 소스 수집

In [1]:
import requests
url = "https://news.daum.net/ranking/popular?regDate=20211015"
agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
response = requests.get(url = url, headers = {"User-Agent":agent})

#### response 객체

In [2]:
print(type(response))

<class 'requests.models.Response'>


In [3]:
print(response.status_code)

200


In [4]:
res_text = response.text
res_content = response.content

print(res_text[:200])
print('=' * 200)
print(res_content[:200])


<!DOCTYPE html>



<html lang="ko" class="os_windows chrome pc version_96_0_4664_45 ">
<head>
<meta charset="utf-8">
<meta name="referrer" content="always" />

<meta property="og:author" content="Dau
b'\n<!DOCTYPE html>\n\n\n\n<html lang="ko" class="os_windows chrome pc version_96_0_4664_45 ">\n<head>\n<meta charset="utf-8">\n<meta name="referrer" content="always" />\n\n<meta property="og:author" content="Dau'


In [5]:
print(type(res_text)) # <class "str">
print(type(res_content)) # <class "bytes">

<class 'str'>
<class 'bytes'>


### BeautifulSoup을 이용한 파싱

In [6]:
from bs4 import BeautifulSoup as BS

In [7]:
parsed_res_text = BS(res_text)
print(type(parsed_res_text))

<class 'bs4.BeautifulSoup'>


#### find와 find_all 메서드

In [8]:
# 첫 번째 ul 태그 찾기
print(parsed_res_text.find("ul"))

<ul data-tiara-layer="GNB default" id="kakaoRelServices">
<li><a class="link_services link_services1" data-tiara-layer="service_enter" href="https://entertain.daum.net"><span class="ir_wa">연예</span></a></li>
<li><a class="link_services link_services2" data-tiara-layer="service_sports" href="https://sports.daum.net"><span class="ir_wa">스포츠</span></a></li>
</ul>


In [9]:
# 세 번째 위치의 ul 태그 찾기
print(parsed_res_text.find_all("ul")[3])

<ul class="list_weather">
<li>
<a class="link_cont" data-tiara-layer="GNB default widget_weather" href="http://search.daum.net/search?w=tot&amp;DA=23W&amp;rtmaxcoll=Z8T&amp;q=서울특별시날씨">
<span class="ico_weather ico_weather1">맑음</span>
<span class="txt_weather">
											서울
											<span class="num_heat">1</span> ℃
                                		</span>
</a>
</li>
<li>
<a class="link_cont" data-tiara-layer="GNB default widget_weather" href="http://search.daum.net/search?w=tot&amp;DA=23W&amp;rtmaxcoll=Z8T&amp;q=수원시 권선구날씨">
<span class="ico_weather ico_weather1">맑음</span>
<span class="txt_weather">
											수원
											<span class="num_heat">1</span> ℃
                                		</span>
</a>
</li>
<li>
<a class="link_cont" data-tiara-layer="GNB default widget_weather" href="http://search.daum.net/search?w=tot&amp;DA=23W&amp;rtmaxcoll=Z8T&amp;q=인천광역시날씨">
<span class="ico_weather ico_weather1">맑음</span>
<span class="txt_weather">
											인천
											<span class="num_h

In [10]:
# class 속성이 info_news인 첫 번째 span 태그 찾기
print(parsed_res_text.find("span", attrs = {"class":"info_news"}))

<span class="info_news">YTN</span>


In [11]:
# 내용이 조선 일보인 모든 span 태그 찾기
print(parsed_res_text.find_all("span", text = "조선일보"))

[<span class="info_news">조선일보</span>, <span class="info_news">조선일보</span>]


In [12]:
ul_tag = parsed_res_text.find("ul", attrs = {"class":"list_news2"})
print(type(ul_tag))

<class 'bs4.element.Tag'>


In [13]:
li_list = ul_tag.find_all("li")

In [14]:
news_name = li_list[0].find("a", attrs = {"class":"link_txt"})
print(news_name) 

<a class="link_txt" href="https://v.daum.net/v/20211015131304113">[양담소]"딸이 사망한 후 다른여자가 생긴 사위, 혼인신고를 하지 않는 이유는?"</a>


In [15]:
press_name = li_list[0].find("span", attrs = {"class":"info_news"})
print(press_name) 

<span class="info_news">YTN</span>


#### 속성을 이용한 값 가져오기

In [16]:
print(news_name["href"])

https://v.daum.net/v/20211015131304113


In [17]:
print(news_name.text)

[양담소]"딸이 사망한 후 다른여자가 생긴 사위, 혼인신고를 하지 않는 이유는?"


In [18]:
print(press_name.text)

YTN


In [19]:
import pandas as pd 
# 빈 리스트로 초기화
news_name_list = []
news_link_list = []
press_name_list = []
for li in li_list: # li를 li_list를 순회하면서
    # class가 link_txt인 a 태그를 news_name_tag에 저장
    news_name_tag = li.find("a", attrs = {"class":"link_txt"})
    press_name_tag = li.find("span", attrs = {"class":"info_news"})

    news_name = news_name_tag.text
    press_name = press_name_tag.text
    news_link = news_name_tag["href"]

    news_name_list.append(news_name)
    press_name_list.append(press_name)
    news_link_list.append(news_link)

data = pd.DataFrame({"뉴스명":news_name_list, 
                    "언론사명":press_name_list,
                    "뉴스링크":news_link_list})

display(data.head(10)) 

,뉴스명,언론사명,뉴스링크
0,"[양담소]""딸이 사망한 후 다른여자가 생긴 사위, 혼인신고를 하지 않는 이유는?""",YTN,https://v.daum.net/v/20211015131304113
1,"""고속도로에서 주저없이 내렸던 형님, 평소 형님다운 행동이었다""",노컷뉴스,https://v.daum.net/v/20211015120306826
2,커지는 'S의 공포'..이미 진행중? 아직 이른 우려?,뉴스1,https://v.daum.net/v/20211015050509768
3,"'석열이형TV'도 나왔던 서민 ""尹에 실망, 추미애씨 사과드린다""",중앙일보,https://v.daum.net/v/20211015095114087
4,교회서 찬송가 불러도.. 윤석열과 부인·장모에 드리운 역술·무속 그림자,오마이뉴스,https://v.daum.net/v/20211015160004414
5,"북한군에 수면제 먹이고 탈북..분노한 김정은 ""무조건 잡아라""",서울신문,https://v.daum.net/v/20211014224612454
6,"북한군에 수면제 먹이고 일가족 탈북..김정은 ""억만금 들여서라도 잡아라""",조선일보,https://v.daum.net/v/20211015064320833
7,"""집값 떨어지기 전 이런 현상 꼭 나옵니다"" 전문가의 조언",머니투데이,https://v.daum.net/v/20211015053300134
8,'오징어 게임' 왜 이러지?.. 미국 언론들의 고민 [최현정의 웰컴 투 아메리카],오마이뉴스,https://v.daum.net/v/20211015071800227
9,"이재명 ""윤석열, 친일파가 독립군 행세..정치활동 중단해야""",뉴시스,https://v.daum.net/v/20211015083818504
